In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function, absolute_import, division

In [ ]:
import numpy as np
from wgomoku import GomokuBoard, GomokuTools as gt, roll_out, NH9x9, Heuristics, HeuristicGomokuPolicy, ThreatSearch
import random
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U = \
    1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
BLACK=0
WHITE=1
EDGES=2
STYLE_MIXED=2

In [ ]:
heuristics = Heuristics(kappa=3.0)

### Experimenting with smaller action spaces
Here we look at the board (states) that all evolve from an initial board. We're going to train on all board snapshots of a series of heuristically played games.

In [ ]:
def new_initial_state(heuristics):
    board = GomokuBoard(heuristics, N=15, disp_width=8)
    policy = HeuristicGomokuPolicy(style = 2, bias=.5, topn=5, 
                                   threat_search=ThreatSearch(2,2))

    board.set(H,8).set('G',6).set(G,8).set(F,8).set(H,9).set(H,10)    
    return board, policy

In [ ]:
new_initial_state(heuristics)[0].display('current')

In [ ]:
board, policy = new_initial_state(heuristics)
_ = roll_out(board, policy, 40)
board.display('current')

In [ ]:
stones = board.stones.copy()

In [ ]:
from wgomoku import heuristic_QF, transform, create_sample, wrap_sample

In [ ]:
sample = create_sample(board.stones.copy(), board.N, 1-board.current_color)

In [ ]:
o,d = np.rollaxis(sample, 2, 0)
2 * d + o

In [ ]:
def create_samples_and_qvalues(board, heuristics):
    all_stones_t = [transform(board.stones.copy(), board.N, rot, ref) 
                    for rot in range(4)
                    for ref in [False, True]]

    samples = []
    qvalues = []
    for stones_t in all_stones_t:
        sample = create_sample(stones_t, board.N, 1-board.current_color)
        board = GomokuBoard(heuristics=heuristics, N=board.N, stones=stones_t)
        policy = HeuristicGomokuPolicy(style = STYLE_MIXED, bias=.5, topn=5, 
                                       threat_search=ThreatSearch(2,2))

        qvalue, default_value = heuristic_QF(board, policy)
        qvalue = wrap_sample(qvalue, default_value)
        samples.append(sample)
        qvalues.append(qvalue)

    return np.array(samples), np.reshape(qvalues, [8, board.N+2, board.N+2, 1])

In [ ]:
samples, values = create_samples_and_qvalues(board, heuristics)

In [ ]:
np.shape(samples),np.shape(values)

In [ ]:
b, w = np.rollaxis(samples[3], 2, 0)
b + 2*w

### Plausibility check

In [ ]:
index = np.argmax(values[3])
r, c = np.divmod(index,17)
pos=gt.m2b((r, c), 17)
print(r,c,pos, values[3][r][c])

In [ ]:
from wgomoku import create_samples_and_qvalues

In [ ]:
def data_from_game(board, policy, heuristics):    
    """
    Careful: This function rolls back the board
    """
    s,v = create_samples_and_qvalues(board, heuristics)
    while board.cursor > 6:
        board.undo()
        s1, v1 = create_samples_and_qvalues(board, heuristics)
        s = np.concatenate((s,s1))
        v = np.concatenate((v,v1))
    return s,v

In [ ]:
from wgomoku import data_from_game

In [ ]:
data = data_from_game(board, policy, heuristics)

In [ ]:
o,d = np.rollaxis(data[0][30], 2, 0)
2 * d + o

#### Yet another plausibility check

In [ ]:
sample_no = 30
index = np.argmax(data[1][sample_no])
r, c = np.divmod(index,17)
pos=gt.m2b((r, c), 17)
print(r,c,pos)
print("Value of the suggested pos:") 
print(data[1][sample_no][r][c])
print("Value to the right of the suggested pos:") 
print(data[1][sample_no][r][c+1])

## Save the results for later training

In [ ]:
np.save("samples.npy", data[0])
np.save("values.npy", data[1])

In [ ]:
samples = np.load("samples.npy")
values = np.load("values.npy")

In [ ]:
samples.shape, values.shape

In [ ]:
np.rollaxis(samples[0], 2, 0)

In [ ]:
np.rollaxis(values[0], 2, 0)

In [ ]:
np.rollaxis(samples[0], 2, 0)[0] + 2*np.rollaxis(samples[0], 2, 0)[1]